<a href="https://colab.research.google.com/github/kuakeke/113-2programming/blob/main/2025%E5%B9%B44%E6%9C%8817%E6%97%A5%20%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%20%E4%BD%9C%E6%A5%AD%E4%B8%89%20%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

柯昱葶 國文115級 41120122L
程式語言 作業三 資料分析

# 作業目標

* 使用 Python 進行資料分析，利用 KMeans 聚類與 PCA 降維，並使用視覺化工具展示結果。資料需替換成自己的資料來進行分析。

# 作業步驟
1. 資料準備：
找到或生成自己的資料，例如：學生的學科成績、活動參與情況等。

2. 資料處理：
清理資料並整理成適合分析的格式，計算每個學生的行為指標或其他指標。

3. 聚類分析：
使用 KMeans 將資料進行聚類，將學生分成不同群組，並針對生成之圖表進行解釋。

4. 降維與視覺化：
使用 PCA 降維後，畫出 2D 圖表來展示群組分佈情況，並針對生成之圖表進行解釋。

5. 提交作業：
在github README 檔案中簡單描述分析過程與視覺化結果。

# 作業分數規定：

* 資料處理 (B-)：只完成資料清理與處理，分析步驟不完全。

* 聚類分析 (B+)：進行聚類分析，群組清晰但解釋略不足。

* PCA降維與視覺化 (A)：資料函多成因，完成降維與視覺化，並且圖表清晰。

* 解釋圖表 (A+)：詳細解釋圖表，能清楚指出關鍵見解。



---



**學生成績**

https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing').sheet1

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,question,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
0,Q1,1,1,1,1,1,1,0,1,0,1
1,Q2,1,1,1,1,1,1,1,1,0,1
2,Q3,1,1,1,1,1,1,0,1,0,1
3,Q4,0,1,1,0,1,1,1,1,0,0
4,Q5,1,1,1,1,1,1,0,1,0,1


In [ ]:
df = df.set_index("question")

# 轉置，變成每列一個學生
df_t = df.T.copy()
df_t.head()


question,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
S1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
S2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
S3,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
S4,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0
S5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1


In [ ]:
# 計算每位學生總分（或平均）
df_t["score"] = df_t.sum(axis=1)

# 依總分排序
sorted_students = df_t.sort_values("score", ascending=False)
n = len(sorted_students)
group_size = round(n * 0.27)
# 分組
high_group = sorted_students.head(group_size).drop(columns="score")
low_group = sorted_students.tail(group_size).drop(columns="score")

# 計算每題高分組與低分組的正答率
high_correct_rate = high_group.T.mean(axis=1)
low_correct_rate = low_group.T.mean(axis=1)

# 計算每題的難度（高+低平均）
difficulty = (high_correct_rate + low_correct_rate) / 2

# 四捨五入到小數點後兩位
difficulty = difficulty.round(2)

# 評鑑描述函式
def get_evaluation_label(p):
    if p >= 0.80:
        return "極容易"
    elif 0.60 <= p < 0.80:
        return "容易"
    elif 0.40 <= p < 0.60:
        return "難易適中"
    elif 0.20 <= p < 0.40:
        return "困難"
    else:
        return "極困難"

# 套用評鑑描述
P = difficulty.apply(get_evaluation_label)

In [ ]:
# 計算每題高分組與低分組的正答率
high_correct_counts = high_group.T.sum(axis=1)
low_correct_counts = low_group.T.sum(axis=1)

# 計算鑑別度
discrimination = (high_correct_counts - low_correct_counts) / (n / 2)

# 四捨五入到小數點後兩位
discrimination = discrimination.round(2)

# 評鑑描述函式
def get_evaluation_label(d):
    if d >= 0.40:
        return "極佳的試題"
    elif 0.30 <= d < 0.40:
        return "尚可的試題，可能需要稍加改進"
    elif 0.20 <= d < 0.30:
        return "不佳的試題，必須加以改進，或棄卻"
    else:
        return "極差的試題，應棄卻"

# 套用評鑑描述
evaluation = discrimination.apply(get_evaluation_label)


In [ ]:
# 合併成 DataFrame
d_df = pd.DataFrame({
    "高分組答對率": high_correct_rate,
    "高分組答對數": high_correct_counts,
    "低分組答對率": low_correct_rate,
    "低分組答對數": low_correct_counts,
    "難度": difficulty,
    "鑑別度": discrimination,
    "鑑別評鑑": evaluation,
    "難易評鑑": P
})

print(d_df)

            高分組答對率  高分組答對數    低分組答對率  低分組答對數    難度  鑑別度              鑑別評鑑  \
question                                                                    
Q1        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q2        1.000000       3  0.666667       2  0.83  0.2  不佳的試題，必須加以改進，或棄卻   
Q3        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q4        0.666667       2  0.666667       2  0.67  0.0         極差的試題，應棄卻   
Q5        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q6        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q7        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q8        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q9        1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q10       1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q11       1.000000       3  0.000000       0  0.50  0.6             極佳的試題   



---


我做了2種k-mean和pca。以下第一種是設置難度和鑑別度兩種的平均值，讓pca去降維使用。

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# 建立難度與鑑別度平均資料
question_summary = d_df.groupby('question')[['難度', '鑑別度']].mean().reset_index()

# 加入 KMeans 聚類
kmeans = KMeans(n_clusters=4, random_state=42)
question_summary['分類群組'] = kmeans.fit_predict(question_summary[['難度', '鑑別度']])

# PCA 降維（可視化用，可不顯示）
pca = PCA(n_components=2)
components = pca.fit_transform(question_summary[['難度', '鑑別度']])
question_summary['主成分1'] = components[:, 0]
question_summary['主成分2'] = components[:, 1]

# 合併相同難度＆鑑別度的題目名稱
grouped = question_summary.groupby(['主成分1', '主成分2', '分類群組']).agg({
    'question': lambda x: ', '.join(x.astype(str))
}).reset_index()


# 將合併後資料與原本的分類群組對齊
fig = px.scatter(
    grouped,
    x='主成分1',
    y='主成分2',
    color='分類群組',
    text='question',
    title='試題分類難度鑑別度聚類結果',
    labels={'鑑別度': '鑑別度', '難度': '難度'},
    hover_data=['question']
)

# 顯示題號標籤
fig.update_traces(textposition='top center')

# 美化圖表
fig.update_layout(
    xaxis_title="主成分1",
    yaxis_title="主成分2",
    legend_title="分類群組",
    title_x=0.5,
    height=600,
    width=800
)

fig.show()


從以上第一種k-mean圖，若以主成分1（x軸）來看，可看見Q19這題聚類位置比其他來得遠。代表題目比較特殊，全班同學都答錯，證明這是一個可以淘汰的題目。而“Q1、Q3、Q5、Q6、Q7、Q8、Q9、Q10、Q12、Q14”被歸為一組，他們共同的特性是難度容易且適中，鑑別度極佳的試卷。



---

第二種圖如下，我做了調整，將難度和鑑別度加入x軸和y軸。整個圖表更清晰了，能明顯看出難度和鑑別度的關係。

In [ ]:
# 建立難度與鑑別度平均資料
question_summary = d_df.groupby('question')[['難度', '鑑別度']].mean().reset_index()

# 根據難度與鑑別度建立中位數分類
diff_median = question_summary['鑑別度'].median()
difficulty_median = question_summary['難度'].median()

def classify_question(row):
    if row['難度'] >= difficulty_median and row['鑑別度'] >= diff_median:
        return '難度高鑑別高'
    elif row['難度'] >= difficulty_median and row['鑑別度'] < diff_median:
        return '難度高鑑別低'
    elif row['難度'] < difficulty_median and row['鑑別度'] >= diff_median:
        return '難度低鑑別高'
    else:
        return '難度低鑑別低'

# 加入分類欄位
question_summary['分類群組'] = question_summary.apply(classify_question, axis=1)

# 加入 KMeans 聚類
kmeans = KMeans(n_clusters=4, random_state=42)
question_summary['分類群組'] = kmeans.fit_predict(question_summary[['難度', '鑑別度']])

# PCA 降維（可視化用，可不顯示）
pca = PCA(n_components=2)
components = pca.fit_transform(question_summary[['難度', '鑑別度']])
question_summary['主成分1'] = components[:, 0]
question_summary['主成分2'] = components[:, 1]

# 合併相同難度＆鑑別度的題目名稱
grouped = question_summary.groupby(['難度', '鑑別度', '分類群組']).agg({
    'question': lambda x: ', '.join(x.astype(str))
}).reset_index()

# 將合併後資料與原本的分類群組對齊
fig = px.scatter(
    grouped,
    x='難度',
    y='鑑別度',
    color='分類群組',
    text='question',
    title='試題分類分析圖（難度和鑑別度）',
    labels={'鑑別度': '鑑別度', '難度': '難度'},
    hover_data=['question']
)

# 顯示題號標籤
fig.update_traces(textposition='top center')

# 美化圖表
fig.update_layout(
    xaxis_title="難度",
    yaxis_title="鑑別度",
    legend_title="分類群組",
    title_x=0.5,
    height=600,
    width=800
)

fig.show()


我用k-mean聚類和pca降維，將各題的難度和鑑別度的數據結合並且進行聚類。將每個群組以顏色分類得出的分析結果如下：

藍色點：難度高 + 鑑別高（很適合篩選學生）

紅色點：難度高但鑑別低（可能太難無法分出學生程度）

綠色點：難度低但鑑別高（簡單但有效的好題）

紫色點：難度低且鑑別低（多為無效題目，應考慮修改）

這樣身為老師在完成批改一份試卷後，將學生答案寫入這裡，就會被分析出題目的難度和鑑別度，以快速知道哪些題目是好題或是能淘汰的題目。舉例如下：

* 可在圖表看到Q19的難度和鑑別度顯示0，代表這題極難也沒有鑑別度（全班答錯），這是一個無效評量的題目。
* Q2難度0.83代表非常容易，可是太多人答對也表示題目鑑別度低。
* Q11、Q13、Q15、Q17群組是最理想的題目。難度0.5適中，鑑別度在0.6是非常好的狀態。鑑別度高的意思是高分組同學答對率比低分組高，表示題目很明確篩選出會作答和不會作答的同學。